In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, sum as _sum, desc
from delta.tables import DeltaTable

spark = SparkSession.builder.appName("HealthcareMediPulse").getOrCreate()



In [0]:
# Patients
patients_data = [
    ("P001", "Arjun Mehta", 34, "M", "North"),
    ("P002", "Neha Sharma", 29, "F", "South"),
    ("P003", "Rahul Gupta", 40, "M", "East"),
    ("P004", "Sneha Nair", 25, "F", "West")
]
patients_df = spark.createDataFrame(patients_data, ["patient_id", "name", "age", "gender", "region"])


In [0]:
# Hospitals
hospitals_data = [
    ("H001", "City Care", "North"),
    ("H002", "LifePlus", "South"),
    ("H003", "MediHope", "East"),
    ("H004", "CureWell", "West")
]
hospitals_df = spark.createDataFrame(hospitals_data, ["hospital_id", "hospital_name", "region"])


In [0]:
# Appointments Day 1
appointments_day1_data = [
    ("A1001", "P001", "H001", "2024-01-10", "Diabetes", 400, "Completed"),
    ("A1002", "P002", "H002", "2024-01-11", "Flu", 250, "Completed"),
    ("A1003", "P003", "H003", "2024-01-11", "Heart Disease", 1000, "Pending"),
    ("A1004", "P004", "H004", "2024-01-12", "Allergy", 300, "Completed")
]
appointments_df = spark.createDataFrame(
    appointments_day1_data,
    ["appointment_id", "patient_id", "hospital_id", "appointment_date", "diagnosis", "cost", "status"]
)


In [0]:
# Bronze Paths
bronze_path = "/mnt/medipulse/bronze/"

# Write Delta tables
patients_df.write.format("delta").mode("overwrite").save(bronze_path + "patients")
hospitals_df.write.format("delta").mode("overwrite").save(bronze_path + "hospitals")
appointments_df.write.format("delta").mode("overwrite").save(bronze_path + "appointments")

print("✅ Bronze Layer Written Successfully")


✅ Bronze Layer Written Successfully


In [0]:
bronze_patients = spark.read.format("delta").load(bronze_path + "patients")
bronze_hospitals = spark.read.format("delta").load(bronze_path + "hospitals")
bronze_appointments = spark.read.format("delta").load(bronze_path + "appointments")


In [0]:
# Rename hospital region to hospital_region to avoid duplicate column
bronze_hospitals_renamed = bronze_hospitals.withColumnRenamed("region", "hospital_region")

# Join
silver_appointments = (
    clean_appointments
    .join(bronze_patients, "patient_id", "inner")
    .join(bronze_hospitals_renamed, "hospital_id", "inner")
    .withColumn("year", year(col("appointment_date")))
    .withColumn("month", month(col("appointment_date")))
)


In [0]:
silver_path = "/mnt/medipulse/silver/"
silver_appointments.write.format("delta").mode("overwrite").save(silver_path + "appointments")

print("✅ Silver Layer Written Successfully")

✅ Silver Layer Written Successfully


In [0]:
from pyspark.sql.functions import sum as _sum, countDistinct

silver_df = spark.read.format("delta").load(silver_path + "appointments")

# Total revenue per hospital
revenue_per_hospital = silver_df.groupBy("hospital_name").agg(
    _sum("cost").alias("total_revenue")
)

# Total patients per region
patients_per_region = silver_df.groupBy("region").agg(
    _sum("cost").alias("total_revenue"),
    countDistinct("patient_id").alias("total_patients")
)

display(revenue_per_hospital)
display(patients_per_region)


hospital_name,total_revenue
City Care,400
CureWell,300
LifePlus,250


region,total_revenue,total_patients
West,300,1
North,400,1
South,250,1


In [0]:
top_diagnosis = silver_df.groupBy("diagnosis").agg(_sum("cost").alias("total_cost")) \
    .orderBy(desc("total_cost")).limit(3)

gold_path = "/mnt/medipulse/gold/"
revenue_per_hospital.write.format("delta").mode("overwrite").save(gold_path + "revenue_per_hospital")
patients_per_region.write.format("delta").mode("overwrite").save(gold_path + "patients_per_region")
top_diagnosis.write.format("delta").mode("overwrite").save(gold_path + "top_diagnosis")

print("✅ Gold Layer Written Successfully")


✅ Gold Layer Written Successfully


In [0]:

# Appointments Day 2
appointments_day2_data = [
    ("A1005", "P002", "H001", "2024-02-01", "Checkup", 500, "Completed"),
    ("A1006", "P003", "H003", "2024-02-02", "Diabetes", 450, "Completed")
]
appointments_day2_df = spark.createDataFrame(
    appointments_day2_data,
    ["appointment_id", "patient_id", "hospital_id", "appointment_date", "diagnosis", "cost", "status"]
)

In [0]:
# Rename hospital region to hospital_region
bronze_hospitals_renamed = bronze_hospitals.withColumnRenamed("region", "hospital_region")

updated_df = (
    appointments_day2_df.filter(col("status") == "Completed")
    .join(bronze_patients, "patient_id", "inner")  # keeps 'region' for patient
    .join(bronze_hospitals_renamed, "hospital_id", "inner")  # 'hospital_region' instead of 'region'
    .withColumn("year", year(col("appointment_date")))
    .withColumn("month", month(col("appointment_date")))
)


In [0]:
updated_df = (
    appointments_day2_df.filter(col("status") == "Completed")
    .join(bronze_patients, "patient_id", "inner")
    .join(bronze_hospitals, "hospital_id", "inner")
    .drop(bronze_hospitals.region)  # drop hospital region
    .withColumn("year", year(col("appointment_date")))
    .withColumn("month", month(col("appointment_date")))
)


In [0]:
# Time Travel: view gold before incremental
gold_before_increment = spark.read.format("delta").option("versionAsOf", 0).load(gold_path + "revenue_per_hospital")
display(gold_before_increment)

# Vacuum safely (7-day retention)
spark.sql(f"VACUUM delta.`{gold_path}revenue_per_hospital` RETAIN 168 HOURS")

# Optimize + Z-Ordering for hospital_id
spark.sql(f"OPTIMIZE delta.`{silver_path}appointments` ZORDER BY (hospital_id)")

print("✅ Delta Lake Features Applied Successfully")


hospital_name,total_revenue
City Care,400
CureWell,300
LifePlus,250


✅ Delta Lake Features Applied Successfully


In [0]:
# 1️⃣ Total revenue per hospital
display(revenue_per_hospital)

# 2️⃣ Average cost per diagnosis category
avg_cost_diagnosis = silver_df.groupBy("diagnosis").agg((_sum("cost")/countDistinct("appointment_id")).alias("avg_cost"))
display(avg_cost_diagnosis)

# 3️⃣ Number of patients per region
patients_count = silver_df.groupBy("region").agg(countDistinct("patient_id").alias("num_patients"))
display(patients_count)

# 4️⃣ Trend of appointments month-over-month
monthly_trend = silver_df.groupBy("year", "month").count().orderBy("year", "month")
display(monthly_trend)

# 5️⃣ Top 5 most expensive treatments in last 6 months
from pyspark.sql.functions import current_date, datediff
recent_df = silver_df.filter(datediff(current_date(), col("appointment_date")) <= 180)
top_5_expensive = recent_df.orderBy(desc("cost")).limit(5)
display(top_5_expensive)

print("🎉 Healthcare Data Engineering Pipeline Complete!")

hospital_name,total_revenue
City Care,400
CureWell,300
LifePlus,250


diagnosis,avg_cost
Flu,250.0
Diabetes,400.0
Allergy,300.0


region,num_patients
West,1
North,1
South,1


year,month,count
2024,1,3


hospital_id,patient_id,appointment_id,appointment_date,diagnosis,cost,status,name,age,gender,region,hospital_name,hospital_region,year,month


🎉 Healthcare Data Engineering Pipeline Complete!
